# 2.2a Rooftop Solar Model - NS

This notebook models a residential rooftop solar (photovoltaic) system. We'll be simulating the electricity generation of our system based on:
- home & roof properties (location, orientation, angle)
- photovoltaic system characteristics  
- historical weather data

Throughout this notebook, we'll model a rooftop solar system, but the modeling would apply to other PV installations as well.

We'll be relying heavily on a 3rd party library, called pvlib, to do the heavy-lifting in fetching weather data and modeling the system
https://pvlib-python.readthedocs.io/


In [ ]:
# Welcome to your second deep dive lab assignment!

# In this notebook, we're going to  model a residential rooftop solar (photovoltaic) system
# We'll be simulating the electricity generation of our system based on:
#  - home & roof properties (location, orientation, angle)
#  - photovoltaic system characteristics
#  - historical weather data

# Throughout this notebook, we'll model a rooftop solar system, but the modeling would apply to other PV installations as well.

# We'll be relying heavily on a 3rd party library, called pvlib, to do the heavy-lifting in fetching weather data and modeling the system
# https://pvlib-python.readthedocs.io/


In [ ]:
# Photovoltaic (PV) panels: the technical term for what we typcially refer to as "solar panels" 
# Wikipedia: "Photovoltaics is the conversion of light into electricity using semiconducting materials that exhibit the photovoltaic effect"
# One other method to harness energy from the sun is "Solar thermal" collection, which concentrates and collects heat from the sun.

# Solar Irradiance: The power per unit area recieved from the Sun (watts per square meter)
#   Direct Normal Irradiance (DNI): Solar radiation that comes directly from the sun
#   Diffuse Horizontal Irradiance (DHI): Solar radiation that does not arrive on a direct path from the sun, e.g. scattered by clouds
#   Global Horizontal Irradiance (GHI): Total solar radiation recieved by a horizonal surface - a mathematical combination of the other two:
#     GHI = DNI * cos(solar zenith angle) + DHI

# System Advisor Model (SAM)
# A public data source, curated by the US National Renewable Energy Laboratory (NREL) agency
# It is "a free techno-economic software model that facilitates decision-making for people in the renewable energy industry"
# Relevant to our interests, it contains detailed models of PV panels and inverters
# https://sam.nrel.gov/

# Physical Solar Model (PSM) a.k.a. solar weather
# Another public data source, also from NREL
# It is "satellite-derived measurements of solar radiation and meteorological data".
# In this notebook we'll call this "solar weather" & it includes solar irradiance and weather data for a particular location throughout the year.
# PSM datasets come in historical (pertaining to a particular year) and TMY (Typical Meterological Year, averaged across several years).
# https://developer.nrel.gov/docs/solar/nsrdb/psm3-2-2-download/


In [ ]:
# Coding Level 0-1:
# Run the existing notebook, but update it to use different home attributes.

# In the next cells we're going to:
# 1. Import 3rd party libraries
# 2. Set input variables (this is where you'll make some changes!)
# 3. Fetch public data: historical solar weather
# 4. Model: Use our `pvlib` library to simulate the electricity output of our particular PV system over a year of historical solar weather


In [ ]:
# In this cell, we're importing packages that we'll use later in the notebook, and defining some shared constants
# You do not need to make changes to this cell.

# Before we can import it, we have to install the pvlib package to the notebook's python kernel
# Hex notebooks have some common 3rd party packages already installed (like pandas), but for less common packages we have to install them first:
%pip install pvlib --quiet

# Imports
from enum import Enum
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pvlib
import pytz
from pathlib import Path
from IPython import display


In [ ]:
# In this cell, we're setting a few input variables to define the location of the system.
# You'll make changes to this cell!

# Location
# Change the two lines below to your home's latitude and longitude. (Find on Google Maps: https://support.google.com/maps/answer/18539)
LATITUDE = 44.927637330292846 #Hopkins,MN // #40.732054681257246 Brooklyn, NY
LONGITUDE = -93.40491129999998 #Hopkins,MN //  #-73.96075940289944 Brooklyn, NY

# Altitude: the altitude of your home (well, technically your roof) above sea level
# Change the line below to match the rough altitude (in meters) of your home (0 = sea level)
ALTITUDE_METERS = 319 # Hopkins, MN // #13m Brooklyn, NY

class Orientation(Enum):
    NORTH = 0
    EAST = 90
    SOUTH = 180
    WEST = 270

# Roof orientation
# Change the line below to be the compass orientation of one side of your roof
# South-facing is best for homes in northern lattitudes; North-facing is best for southern lattitudes
ORIENTATION = Orientation.SOUTH.value


In [ ]:
# In this cell, we're defining the characterics of our desired PV system
# We've picked some reasonble defaults; you can try changing some values if you'd like!

# A PV system is made up of:
#  - an array of one or more PV panels
#  - one or more inverters (electrical devices that convert the DC output from the panels to AC energy)

# We'll model a system with a single panel and a single inverter. We'll scale up that modeled capacity to match our target system capacity.

# PV System Capacity
# The capacity is the maximum energy your system could output in ideal sun conditions
# The capacity is limited by the size of your roof area - you get higher capacity by fitting in more PV panels
# For a single-family home, a very large system would have a capacity of 15kW (15000 Watts)
PV_SYSTEM_CAPACITY_WATTS = 5000  # default 5000

# The `pvlib` 3rd party library fetches detailed models of a variety of PV panels and inverters, from the NREL SAM (see definition above)
# We've arbitrarily selected one of each
PV_PANEL_MODEL = pvlib.pvsystem.retrieve_sam("SandiaMod")["Canadian_Solar_CS5P_220M___2009_"]
PV_PANEL_CAPACITY_WATTS = 220  # default 220
INVERTER_MODEL = pvlib.pvsystem.retrieve_sam("cecinverter")["ABB__MICRO_0_25_I_OUTD_US_208__208V_"]

# Our scaling factor - how many panels of {PV_PANEL_CAPACITY_WATTS} we need to reach a total system capacity of {PV_SYSTEM_CAPACITY_WATTS}
# (we're letting this be a fractional number)
NUMBER_OF_PANELS = PV_SYSTEM_CAPACITY_WATTS / PV_PANEL_CAPACITY_WATTS

# Azimuth: the compass rose orientation of the panels
# We've set this to match roof orientation defined above, but you could pick something else
PV_ARRAY_AZIMUTH = ORIENTATION

# Tilt: how the panels are tilted relative to the earth's surface
# Typically, ideal (fixed) tilt angle is equal to your latitude
# For example, at the equator (latitude 0 degrees), panels should face straight up (0 degrees) to get the most sun throughout the year
PV_ARRAY_TILT = LATITUDE

# Weather Data Simulation Year
# We're going to simulate our solar system's output over a year, using historical weather data for an actual year in the past
# 2022 is the most recent year for which we can get historical solar weather data from NREL (but you could choose an earlier year)
SIMULATION_YEAR = 2022


In [ ]:
# In this cell, we're using `pvlib` to fetch historical "solar weather" data for our chosen location over our chosen simulation year
# "Solar weather" is how much sun we got at this location

# Note: You'll need to set your NREL API credentials
# Get them from: https://developer.nrel.gov/signup/
NREL_API_KEY = "YOUR_NREL_API_KEY_HERE"  # Replace with your actual API key
NREL_API_EMAIL = "YOUR_EMAIL_HERE"  # Replace with your actual email

solar_weather_timeseries, solar_weather_metadata = pvlib.iotools.get_psm3(
    latitude=LATITUDE,
    longitude=LONGITUDE,
    names=SIMULATION_YEAR,
    api_key=NREL_API_KEY,
    email=NREL_API_EMAIL,
    map_variables=True,
    leap_day=True,
)
solar_weather_timeseries


In [ ]:
# In this cell, we're putting it all together!
# We're using `pvlib` to simulate how much electricity our PV system would generate given historical "solar weather" data

def simulate_pv_ouptput(
    solar_weather_timeseries,
    latitude,
    longitude,
    altitude,
    pv_array_tilt,
    pv_array_azimuth,
    pv_panel_model,
    inverter_model,
):
    # Adapted from example: https://pvlib-python.readthedocs.io/en/v0.9.0/introtutorial.html?highlight=total_irradiance#procedural

    # First, we model the position of the sun relative to our chosen location over the simulation year
    solar_position_timeseries = pvlib.solarposition.get_solarposition(
        time=solar_weather_timeseries.index,
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        temperature=solar_weather_timeseries["temp_air"],
    )

    # We combine solar position with historical solar weather data to model total irradiance for our PV panel
    total_irradiance_timeseries = pvlib.irradiance.get_total_irradiance(
        pv_array_tilt,
        pv_array_azimuth,
        solar_position_timeseries["apparent_zenith"],
        solar_position_timeseries["azimuth"],
        solar_weather_timeseries["dni"],
        solar_weather_timeseries["ghi"],
        solar_weather_timeseries["dhi"],
        dni_extra=pvlib.irradiance.get_extra_radiation(solar_weather_timeseries.index),
        model="haydavies",
    )

    # We then model air mass & angle of incidence, which we combine with total irradiance to model "effective" irradiance on our PV panel
    # Air mass is a measure of the path length of solar radiation through the atmosphere
    absolute_airmass_timeseries = pvlib.atmosphere.get_absolute_airmass(
        pvlib.atmosphere.get_relative_airmass(
            solar_position_timeseries["apparent_zenith"]
        ),
        pvlib.atmosphere.alt2pres(ALTITUDE_METERS),
    )

    # Angle of incidence is the angle of the sun's rays relative to the panel's surface
    angle_of_incidence_timeseries = pvlib.irradiance.aoi(
        pv_array_tilt,
        pv_array_azimuth,
        solar_position_timeseries["apparent_zenith"],
        solar_position_timeseries["azimuth"],
    )

    # This is where we combine the direct and diffuse irradiance, taking into account the air mass that the sunlight has to travel through
    effective_irradiance_timeseries = pvlib.pvsystem.sapm_effective_irradiance(
        total_irradiance_timeseries["poa_direct"],
        total_irradiance_timeseries["poa_diffuse"],
        absolute_airmass_timeseries,
        angle_of_incidence_timeseries,
        pv_panel_model,
    )

    # We model the temperature within the PV panel ("cell temperature"), which affects the efficiency of the panels
    cell_temperature_timeseries = pvlib.temperature.sapm_cell(
        total_irradiance_timeseries["poa_global"],
        solar_weather_timeseries["temp_air"],
        solar_weather_timeseries["wind_speed"],
        **pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS["sapm"]["open_rack_glass_glass"],
    )

    # Finally we put it all together:

    # We simulate the DC electricity output of our PV panel given the effective solar irradiance and cell temperature)
    dc_electricity_timeseries = pvlib.pvsystem.sapm(
        effective_irradiance_timeseries, 
        cell_temperature_timeseries, 
        pv_panel_model
    )

    # And then we simulate the inverter converting the DC output into AC output
    ac_electricity_timeseries_watts = pvlib.inverter.sandia(
        dc_electricity_timeseries["v_mp"], 
        dc_electricity_timeseries["p_mp"], 
        inverter_model
    )

    # Wrap the results all up into a dataframe for plotting!
    pv_model_results = pd.DataFrame(
        {
            "PV Array Output (Wh)": dc_electricity_timeseries["i_mp"] * dc_electricity_timeseries["v_mp"] * NUMBER_OF_PANELS,
            "Inverter Output (Wh)": ac_electricity_timeseries_watts * NUMBER_OF_PANELS,
            "Solar azimuth (°)": solar_position_timeseries["azimuth"],
            "Solar elevation (°)": solar_position_timeseries["apparent_elevation"],
        }
    )
    pv_model_results["timestamp"] = pv_model_results.index
    return pv_model_results

PV_SYSTEM_CHARACTERISTICS = {
    "latitude": LATITUDE,
    "longitude": LONGITUDE,
    "altitude": ALTITUDE_METERS,
    "pv_array_tilt": PV_ARRAY_TILT,
    "pv_array_azimuth": PV_ARRAY_AZIMUTH,
    "pv_panel_model": PV_PANEL_MODEL,
    "inverter_model": INVERTER_MODEL,
}

pv_model_results = simulate_pv_ouptput(solar_weather_timeseries, **PV_SYSTEM_CHARACTERISTICS)
pv_model_results

# Explain Code: The double asterisks (**) before PV_SYSTEM_CHARACTERISTICS are used for dictionary unpacking. 
# It takes a dictionary (PV_SYSTEM_CHARACTERISTICS) and passes its key-value pairs as keyword arguments to the function. 
# This assumes that PV_SYSTEM_CHARACTERISTICS contains parameters needed by the simulate_pv_output function.


## Weekly PV Generation - entire year

**Note:** The original Hex notebook had an interactive chart here. Here's a matplotlib visualization:


In [ ]:
# Create a weekly aggregated plot of PV generation
fig, ax = plt.subplots(figsize=(12, 6))

if not pv_model_results.empty:
    # Convert to weekly data
    weekly_data = pv_model_results.resample('W')['Inverter Output (Wh)'].sum()
    
    # Create area plot
    ax.fill_between(weekly_data.index, weekly_data.values, color='#EECA3B', alpha=0.7)
    ax.plot(weekly_data.index, weekly_data.values, color='#EECA3B', linewidth=2)
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Generation (Wh)')
    ax.set_title(f'Weekly PV Generation - {SIMULATION_YEAR}')
    ax.grid(True, alpha=0.3)
    
    # Format x-axis
    ax.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator(interval=2))
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot. Please check your API credentials and data connection.")


## 1 week of modeled solar output

**Note:** The original Hex notebook had an interactive chart here. Here's a matplotlib visualization showing both solar output and solar elevation:


In [ ]:
# Create a detailed plot for one week of data
fig, ax1 = plt.subplots(figsize=(14, 8))

if not pv_model_results.empty:
    # Select one week of data (March 7-14, 2022 as in original)
    start_date = '2022-03-07'
    end_date = '2022-03-14'
    week_data = pv_model_results.loc[start_date:end_date]
    
    if not week_data.empty:
        # Create primary y-axis for solar output
        color1 = '#EECA3B'
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Inverter Output (Wh)', color=color1)
        ax1.fill_between(week_data.index, week_data['Inverter Output (Wh)'], 
                        color=color1, alpha=0.7, label='Solar Output')
        ax1.plot(week_data.index, week_data['Inverter Output (Wh)'], 
                color=color1, linewidth=2)
        ax1.tick_params(axis='y', labelcolor=color1)
        ax1.grid(True, alpha=0.3)
        
        # Create secondary y-axis for solar elevation
        ax2 = ax1.twinx()
        color2 = '#4C78A8'
        ax2.set_ylabel('Solar Elevation (°)', color=color2)
        ax2.plot(week_data.index, week_data['Solar elevation (°)'], 
                color=color2, linewidth=2, label='Solar Elevation')
        ax2.tick_params(axis='y', labelcolor=color2)
        
        # Format x-axis
        ax1.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m-%d %H:%M'))
        ax1.xaxis.set_major_locator(plt.matplotlib.dates.HourLocator(interval=12))
        plt.xticks(rotation=45)
        
        ax1.set_title(f'1 Week of Modeled Solar Output - {start_date} to {end_date}')
        
        # Add legends
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
        plt.tight_layout()
        plt.show()
    else:
        print(f"No data available for the selected week ({start_date} to {end_date})")
else:
    print("No data available to plot. Please check your API credentials and data connection.")


In [ ]:
average_usage_kwh = 10791 # Average yearly electricity usage for a US residential home, in 2022 (https://www.eia.gov/tools/faqs/faq.php)
total_output_kWh = pv_model_results['Inverter Output (Wh)'].sum() / 1000

print(f"In {SIMULATION_YEAR}, our system would have produced:")
print(f"{total_output_kWh:.0f} kWh ({total_output_kWh/average_usage_kwh*100:.0f}% of average US residential usage)")


In [ ]:
# If you're at coding level 0 or 1: Congrats, you made it through the code!

# Now, go back and try tweaking some input variables to play around with the data and look for interesting findings. Ideas:

#   - How significant is the effect of home altitude on the output?
###### When I compare my initial setup at 13m, then 0m, then 313m it seems to have little effect on output (1-10kWh on the year).
###### A google search tells me otherwise, that high altitude can have up to 28% increase in efficiency over ground installs.
###### Perhaps my location in Brooklyn, NY is what makes altitude less of a factor?

#   - What is the impact of orienting the array South-facing vs North-facing?
###### My Brooklyn, NY setup is heavily impacted by facing the panels North instead of South, dropping output to 3285 kWh from 7573 kWh (57% drop).
###### Unless panels were situated near the equator, I would expect flipping the orientation would greatly impact setup ouput in most lattitudes.

#   - How does output of the same PV system compare between two different locations?
###### Switching location from Brooklyn, NY to Hopkins, MN surprisingly increased to annual output to 7704 kWh from 7573 kWh.
###### Minnesota had more volitile output throughout the year, but ultimately output spikes early in the year resulted in greater overall output.
###### I expect the estimated elevation change (to 319m from 13m) is a factor in the occassional increased efficiency seen in Minnesota.

#   - Look up your yearly electricity usage. What capacity system would you need to cover your consumption?
#     (assuming net metering, where you just have to put as much energy in over the course of the year as you take out)
##### With an estimated average of 300 kWh monthly consupmtion in my NY apartment, I would need an annual system capacity of 3600 kWh.

# In your Assignment submission on the Terra.do app:
#  1. Link to your copy of this notebook
#  2. Write up a few sentences summarizing your interesting finding (you can share this in slack as well!)


In [ ]:
# Coding Level 2-3 - Carry on!

# If you have strong programming experience, use this as a starting point and modify or expand this notebook. Some ideas:
#  - Fetch TMY data (Typical Meterological Year, averaged across several years) instead of a data from a specific year
#  - Iterate over one or more attributes (programmatically, with charts to compare the outputs)
#  - Update the simulation to combine multiple panels with a single inverter. Does this have a significant effect?
#  - Combine this notebook with the home energy usage notebook to model what capacity PV system you would need to operate off-grid, 
#    i.e. meet your energy usage for each hour of the year, not just in total (accounting for high usage but low generation in winter)
